# 让`deque`支持切片下标

## 使用`itertools.islice()`获取切片

In [1]:
from collections import deque

Python的标准模块`collections`中的`deque`是一个双向链表结构，它支持在该链表的左右添加元素或者删除元素。它还支持整数下标运算，但是它不支持切片下标：

In [2]:
d = deque(range(10))

print(d[-1])
print(d[-3:])

9


TypeError: sequence index must be integer, not 'slice'

使用`itertools`模块中的`slice()`可以获得对指定的切片迭代的对象，然后再调用`list()`将迭代器转换为列表即可：

In [3]:
from itertools import islice

list(islice(d, len(d) - 3, len(d), 1))

[7, 8, 9]

但是`islice()`不支持负数下标，可以使用`slice`对象的`indices()`方法将一个切片对象转换为`start`、`end`和`step`三个整数值。例如：

In [4]:
start, end, step = slice(-3, None).indices(len(d))
list(islice(d, start, end, step))

[7, 8, 9]

如上所述，我们知道了如何使用切片获取`deque`对象的部分元素，下面让我们看看如何让`deque`支持切片下标。

## 切片代码是如何运行的

首先使用`dis.dis()`查看`a[1:2]`编译之后的代码，可以看到与其对应的操作符为`BINARY_SUBSCR`。

In [9]:
from dis import dis

def test_slice():
    a = [1, 2, 3]
    b = a[1:2]
    
dis(test_slice)

  4           0 LOAD_CONST               1 (1)
              3 LOAD_CONST               2 (2)
              6 LOAD_CONST               3 (3)
              9 BUILD_LIST               3
             12 STORE_FAST               0 (a)

  5          15 LOAD_FAST                0 (a)
             18 LOAD_CONST               1 (1)
             21 LOAD_CONST               2 (2)
             24 BUILD_SLICE              2
             27 BINARY_SUBSCR
             28 STORE_FAST               1 (b)
             31 LOAD_CONST               0 (None)
             34 RETURN_VALUE


在`https://github.com/python/cpython`中搜索`BINARY_SUBSCR`可以在`ceval.c`中找到如下代码：

```c
TARGET(BINARY_SUBSCR) {
    PyObject *sub = POP();
    PyObject *container = TOP();
    PyObject *res = PyObject_GetItem(container, sub);
    Py_DECREF(container);
    Py_DECREF(sub);
    SET_TOP(res);
```

接着可以在`abstract.c`中搜索到`PyObject_GetItem()`的定义：

```c
PyObject *
PyObject_GetItem(PyObject *o, PyObject *key)
{
    PyMappingMethods *m;

    if (o == NULL || key == NULL) {
        return null_error();
    }

    m = o->ob_type->tp_as_mapping;
    if (m && m->mp_subscript) {
        PyObject *item = m->mp_subscript(o, key);
        assert((item != NULL) ^ (PyErr_Occurred() != NULL));
        return item;
    }

    if (o->ob_type->tp_as_sequence) {
        if (PyIndex_Check(key)) {
            Py_ssize_t key_value;
            key_value = PyNumber_AsSsize_t(key, PyExc_IndexError);
            if (key_value == -1 && PyErr_Occurred())
                return NULL;
            return PySequence_GetItem(o, key_value);
        }
        else if (o->ob_type->tp_as_sequence->sq_item)
            return type_error("sequence index must "
                              "be integer, not '%.200s'", key);
    }

    return type_error("'%.200s' object is not subscriptable", o);
}
```

可以看到它会首先尝试`o->ob_type->tp_as_mapping`中的`mp_subscript`函数。由于`deque`没有定义`tp_as_mapping`，因此不支持切片运算。我们可以参考列表对象的代码`listobject.c`中的实现。下面使用`Cython`实现`mp_subscript`函数。

## 使用`cython`和`cffi`让`deque`支持切片

In [1]:
%load_ext cython

`tp_as_mapping`指向一个有三个函数指针的`PyMappingMethods`结构体：

* `mp_length(o)`: 返回对象`o`的长度
* `mp_subscript(o, key)`: 返回`o[key]`的运算结果
* `mp_ass_subscript`: 本例中不需要使用该函数，省略

`addr_of_mapping_methods()`返回全局变量`deque_mapping_methods`的地址，我们需要把该地址写入`deque`对应的结构体中的`tp_as_mapping`字段。

In [3]:
%%cython
cdef extern from "object.h":
    ctypedef struct PyMappingMethods:
        void * mp_length
        void * mp_subscript
        void * mp_ass_subscript
    
from cpython.ref cimport PyObject
from cpython.sequence cimport PySequence_GetItem
from cpython.slice cimport PySlice_GetIndices
from itertools import islice
from collections import deque

cdef ssize_t deque_len(object dq):
    return len(object)

cdef deque_subscript(object dq, object item):
    cdef ssize_t start, end, step
    if isinstance(item, int):
        return PySequence_GetItem(dq, <ssize_t>item)
    elif isinstance(item, slice):
        PySlice_GetIndices(item, len(dq), &start, &end, &step)
        return deque(islice(<object>dq, start, end, step))

cdef PyMappingMethods deque_mapping_methods = [<void *>deque_len, <void *>deque_subscript, NULL]

def addr_of_mapping_methods():
    return <ssize_t>&(deque_mapping_methods)    

下面通过`cffi`的编译功能获得`PyTypeObject`结构体中`tp_as_mapping`字段的偏移量：

In [ ]:
import cffi
ffi = cffi.FFI()

ffi.cdef("""
ssize_t mapping_methods_offset;
""")

lib = ffi.verify("""
ssize_t mapping_methods_offset = offsetof(PyTypeObject, tp_as_mapping);
""")

确认上述代码的计算结果为112:

In [16]:
lib.mapping_methods_offset

112

`deque`在C语言中是一个`PyTypeObject`结构体，下面获取其`tp_as_mapping`字段的地址，并将`deque_mapping_methods`结构体的地址写入该字段：

In [8]:
tp_as_mapping = ffi.cast("ssize_t *", id(deque) + lib.mapping_methods_offset)
tp_as_mapping[0] = addr_of_mapping_methods()

下面可以测试效果了:

In [17]:
a = deque(range(0, 10))
print(a[4:-2])

deque([4, 5, 6, 7])
